In [1]:
from typing import Union, Sequence, Tuple, Optional
from collections import defaultdict
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
import torch
from torch import nn, optim, Tensor
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, Subset

In [2]:
DATA_DIR = Path("../data")
RAW = DATA_DIR / "raw"
INTERIM = DATA_DIR / "interim"
PROCESSED = DATA_DIR / "processed"

# Предобработка данных

На этом шаге
* удалим столбцы, которые не используются в данном задании;
* разделим датасет, выделив данные от самой поздней даты -- 2 октября 2021 года -- в тестовую выборку.

В первом ДЗ у меня возникали проблемы с нехваткой оперативной памяти, поэтому в данном задании я существенно изменил реализацию. Например, при удалении столбцов не используется `pandas`, исходный файл считывается построчно. Естественно, из-за подобных изменений увеличились время выполнения операций и объём используемой памяти.

In [3]:
def drop_cols(file: Union[Path, str],
              file_out: Union[Path, str],
              cols: Sequence[str] = []):
    "Copy csv file without selected columns"
    # helper function for applying mask to a csv file line
    def filter_line(line: str, mask: list):
        line_lst = line.rstrip().split(",")
        filtered = [col for col, drop in zip(line_lst, mask) if not drop]
        return ",".join(filtered) + "\n"

    with open(file, "r") as fin:
        # get boolean mask from header
        header = next(fin)
        header_lst = header.rstrip().split(',')
        mask = [col in cols for col in header_lst]

        # apply mask to every line and export results
        with open(file_out, "w") as fout:
            fout.write(filter_line(header, mask))
            for line in fin:
                fout.write(filter_line(line, mask))


def str_to_date(s: str) -> datetime.date:
    return datetime.fromisoformat(s).date()


def date_split(file: Union[Path, str],
               output_dir: Union[Path, str],
               test_date: str = "2021-10-02",
               date_col: int = 0):
    """
    Split csv file by putting all entries with `test_date` date into test
    dataset. Creates `train.csv` and `test.csv` in `output_dir`.
    """
    output_dir = Path(output_dir)
    assert output_dir.exists()
    test_date = str_to_date(test_date)

    with open(file, "r") as infile:
        # output file objects
        f_train = open(output_dir / "train.csv", "w")
        f_test = open(output_dir / "test.csv", "w")

        # write header
        header = next(infile)
        f_train.write(header)
        f_test.write(header)

        # copy data to one of the files line by line
        for line in infile:
            date_str = line.split(",")[date_col]
            date = str_to_date(date_str)
            fout = f_test if date == test_date else f_train
            fout.write(line)

        f_train.close()
        f_test.close()

In [4]:
drop_cols(file=RAW / "data.csv",
          file_out=INTERIM / "data.csv",
          cols=["banner_id0", "banner_id1", "rate0", "rate1", "g0", "g1",
                "coeff_sum0", "coeff_sum1"])
date_split(file=INTERIM / "data.csv",
           output_dir=INTERIM,
           test_date="2021-10-02",
           date_col=0)

!head {INTERIM / "train.csv"}

date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,impressions,clicks
2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1,1
2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,1
2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,1,1
2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,1,1
2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,1,1
2021-09-27 00:06:50.000000,5,5,2377014068362699676,0,2,2,1,1
2021-09-27 00:07:34.000000,6,6,6863358899511896876,0,3,0,1,1
2021-09-27 00:08:49.000000,7,7,2876502170484631685,0,4,1,1,1
2021-09-27 00:09:08.000000,8,8,5839858970958967275,0,4,3,1,1


# Feature engineering

Здесь выполним такие же преобразования, как и в ДЗ №1. При этом немного поменяем способ хранения категориальных данных: вместо разреженных one-hot массивов будем использовать токены от $0$ до $n$. Как и в прошлом задании будем ограничивать число уникальных токенов (one-hot признаков), объединяя редкие значения.

Изменение вида представления данных вызвано тем, что для реализации FM в этом задании использовался пакет *PyTorch*, в котором разреженные массивы пока поддерживают только ограниченное число операций. Поэтому аналогичная показанной ниже реализация FM, использующая разреженные данные, оказываетcя существенно медленнее (по крайней мере, у меня получилось так).

In [5]:
ONE_HOT_KWARGS = {
    "zone_id": dict(min_frequency=100),
    "banner_id": dict(min_frequency=20),
    "oaid_hash": dict(min_frequency=20),
    "os_id": dict(max_categories=8),
    "country_id": {}
}

In [6]:
def get_tokenizer(data: pd.Series,
                  min_frequency: Optional[int] = None,
                  max_categories: Optional[int] = None
                  ) -> defaultdict:
    """
    Return a mapping from categorical features to integers from 0 to n.
    Order is defined by frequencies, n can be limited by function arguments.
    """
    vc = data.value_counts(sort=True)  # series: value -> count
    if min_frequency is not None:
        if 0 < min_frequency < 1:
            min_frequency *= len(data)
        vc = vc[vc >= min_frequency]
    if max_categories is not None:
        vc = vc.iloc[:max_categories - 1]  # + default value
    n = len(vc)
    return defaultdict(lambda: n, zip(vc.index, range(n)))

In [7]:
def feature_engineering_dense(input_dir: Union[Path, str],
                              output_dir: Union[Path, str]):
    output_dir = Path(output_dir)
    assert output_dir.exists()

    print("Reading data...")
    df_train = pd.read_csv(Path(input_dir) / "train.csv")
    df_test = pd.read_csv(Path(input_dir) / "test.csv")
    dataframes = (df_train, df_test)
 
    # numerical features
    print("Transforming numerical features...")
    for df in dataframes:
        dt = pd.to_datetime(df["date_time"])
        df["weekday"] = dt.dt.weekday
        df["hour"] = dt.dt.hour
        df["log_campaign_clicks"] = np.log(df["campaign_clicks"] + 1)
        df.drop(columns=["date_time", "campaign_clicks", "impressions"],
                inplace=True)

    # categorical features
    print("Transforming categorical features...")
    for name, kwargs in ONE_HOT_KWARGS.items():
        tok = get_tokenizer(df_train[name], **kwargs)
        for df in dataframes:
            unchanged = np.ones(len(df), dtype=np.bool8)
            for value, token in tqdm(tok.items()):
                mask = (df[name] == value) & unchanged
                df.loc[mask, name] = token
                unchanged[mask] = False
            df.loc[unchanged, name] = tok.default_factory()

    # export results
    df_train.to_csv(output_dir / "train.csv", index=False)
    df_test.to_csv(output_dir / "test.csv", index=False)

In [8]:
feature_engineering_dense(INTERIM, PROCESSED)

!head {PROCESSED / "train.csv"}

Reading data...
Transforming numerical features...
Transforming categorical features...


100%|███████████████████████████████████████████| 17/17 [00:00<00:00, 87.33it/s]


zone_id,banner_id,oaid_hash,os_id,country_id,clicks,weekday,hour,log_campaign_clicks
4,14,58846,1,0,1,0,0,0.0
5,291,58846,1,4,1,6,22,0.0
9,8,4525,1,0,1,6,23,1.3862943611198906
19,2,58846,2,4,1,0,0,0.0
206,159,58846,2,0,1,0,0,0.0
115,503,58846,0,16,1,0,0,0.0
85,20,58846,4,0,1,0,0,0.0
94,22,58846,3,4,1,0,0,0.0
18,88,58846,3,5,1,0,0,0.0


# Обучение FM

Класс для работы с преобразованными данными. Категориальные и количественные признаки возвращаются как два отдельных массива (`Xc` и `Xn`, соответственно).

In [9]:
class ClickDatasetTokenized(Dataset):
    """
    Dataset that uses categorical data encoded as tokens {0, 1, .., n}.
    """
    def __init__(
        self,
        file: Union[Path, str],
        categorical: Tuple[str] = \
            ("oaid_hash", "banner_id", "country_id", "os_id", "zone_id"),
        numerical: Tuple[str] = \
            ("weekday", "hour", "log_campaign_clicks"),
        target: str = "clicks"
    ):
        # load dataframe
        df = pd.read_csv(file)
        # categorical features
        self.Xc = np.zeros(shape=(len(df), len(categorical)), dtype=np.int32)
        self.Xc[:, :] = df.loc[:, categorical]
        self.cat_sizes = tuple([len(df[col].unique()) for col in categorical])
        # numerical features
        self.Xn = np.zeros(shape=(len(df), len(numerical)), dtype=np.float32)
        self.Xn[:, :] = df.loc[:, numerical]
        # target
        self.y = df[target].to_numpy().astype(np.float32)

        # save feature names
        self.categorical = categorical
        self.numerical = numerical

    def __len__(self) -> int:
        return self.y.shape[0]

    def __getitem__(self, index) -> Tuple[np.ndarray]:
        return self.Xc[index], self.Xn[index], self.y[index] 

In [10]:
ds_train = ClickDatasetTokenized(PROCESSED / "train.csv")
ds_test = ClickDatasetTokenized(PROCESSED / "test.csv")

Предсказания в FM делаются по формуле
\begin{equation*}
    y_s =
        \theta + \sum_{i=1}^{n}\theta_i x_{is}
        +\sum_{i=1}^{n-1}\sum_{j=i+1}^n
            \langle \mathbf{w}_i, \mathbf{w}_j \rangle x_{is} x_{js}.
\end{equation*}

Следуя [соответствующей главе Dive into Deep Learning](https://d2l.ai/chapter_recommender-systems/fm.html) преобразуем последнее слагаемое
\begin{equation*}
    \sum_{i=1}^{n-1}\sum_{j=i+1}^n
            \langle \mathbf{w}_i, \mathbf{w}_j \rangle x_{is} x_{js}
    =
    \frac{1}{2} \sum_{k=1}^d \left(
        \left(\sum_{i=1}^n{\mathbf{w}_{i,k} x_{is, k}} \right)^2
         - \sum_{i=1}^d{\mathbf{w}_{i,k}^2 x_{is,k}^2}
    \right).
\end{equation*}
Здесь $d$ -- размерность эмбеддингов признаков.

В линейной части модели для категориальных признаков будем вместо весов использовать эмбеддинги размерности $1$. Также для удобства пронумеруем значения всех категориальных признаков. Такое изменение позволяет использовать один модуль `nn.Embedding` для эмбеддингов всех признаков.

In [11]:
class FactorizationMachineTokenized(nn.Module):
    def __init__(self, categorical_sizes: Sequence[int],
                 n_numerical: int, emb_dim: int = 10):
        super().__init__()
        self.linear_c = CategoricalLinear(categorical_sizes)
        self.linear_n = nn.Linear(n_numerical, 1)
        self.emb_c = Embeddings(categorical_sizes, emb_dim)
        self.emb_n = nn.Parameter(torch.randn((n_numerical, emb_dim)))

    def forward(self, Xc: Tensor, Xn: Tensor) -> Tensor:
        linear = self.linear_c(Xc) + self.linear_n(Xn)

        Xn = Xn.unsqueeze(2)
        vx_c = self.emb_c(Xc)  # one-hot => no multiplication
        vx_n = Xn * self.emb_n
        vx = torch.cat([vx_c, vx_n], 1)
        sq_sum = vx.mean(1)**2
        sum_sq = (vx**2).mean(1)

        logits = linear + 0.5 * torch.sum(sq_sum - sum_sq, 1, keepdim=True)
        return logits

    @torch.no_grad()
    def predict(self, dl: DataLoader) -> np.ndarray:
        predictions = []
        for batch in tqdm(dl):
            Xc, Xn = batch[:2]
            output = torch.sigmoid(self.forward(Xc, Xn)).cpu().numpy()
            predictions += list(output.ravel())
        return np.array(predictions, dtype=np.float32)


class Embeddings(nn.Module):
    def __init__(self, emb_numbers: Sequence[int], emb_dim: int):
        super().__init__()
        self.offsets = torch.tensor(np.cumsum(emb_numbers) - emb_numbers[0],
                                    dtype=torch.int32)
        self.emb = nn.Embedding(np.sum(emb_numbers), emb_dim)

    def forward(self, x: Tensor) -> Tensor:
        x = x + self.offsets
        return self.emb(x)


class CategoricalLinear(Embeddings):
    def __init__(self, categorical_sizes: Sequence[int]):
        super().__init__(categorical_sizes, 1)

    def forward(self, x: Tensor) -> Tensor:
        output = super().forward(x)
        return output.mean(1)

Далее обучим модель на полной обучающей выборке. Перед этим был выполнен ряд экспериментов для подбора гиперпараметров. В частности, для выбора размерности эмбеддингов $d$ выполнялся скрипт `cross_validation.py` с $d = 5$, $10$ и $20$. Графики обучения при $d = 5$ приведены ниже, увеличение $d$ не привело к улучшению результатов. Введение регуляризации (`weight_decay`) также не приносило никаких дивидендов.

![train_loss](images/train_loss.png)
![val_loss](images/val_loss.png)

In [12]:
model = FactorizationMachineTokenized(
    ds_train.cat_sizes, len(ds_train.numerical), 5)
model

FactorizationMachineTokenized(
  (linear_c): CategoricalLinear(
    (emb): Embedding(60875, 1)
  )
  (linear_n): Linear(in_features=3, out_features=1, bias=True)
  (emb_c): Embeddings(
    (emb): Embedding(60875, 5)
  )
)

In [13]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
dl_train = DataLoader(ds_train, batch_size=2048, shuffle=True)
dl_test = DataLoader(ds_test, batch_size=4096, shuffle=False)

In [14]:
for epoch in range(3):
    num_iter = 0
    avg_loss = 0
    for Xc, Xn, y in tqdm(dl_train):
        logits = model(Xc, Xn)
        loss = loss_fn(logits, y.unsqueeze(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        num_iter += 1
        avg_loss *= (num_iter - 1) / num_iter
        avg_loss += loss.item() / num_iter
    print(f"Epoch {epoch}: train_loss = {avg_loss:.4f}")
    scheduler.step()

100%|███████████████████████████████████████| 6686/6686 [02:28<00:00, 45.14it/s]


Epoch 0: train_loss = 0.1754


100%|███████████████████████████████████████| 6686/6686 [02:17<00:00, 48.45it/s]


Epoch 1: train_loss = 0.1063


100%|███████████████████████████████████████| 6686/6686 [02:16<00:00, 48.85it/s]

Epoch 2: train_loss = 0.1054


In [15]:
pred = model.predict(dl_test)
test_loss = log_loss(ds_test.y, pred, eps=1e-7)
test_auc = roc_auc_score(ds_test.y, pred)
print(f"Test loss: {test_loss:.4f}, test ROC-AUC: {test_auc: .3f}")

100%|█████████████████████████████████████████| 520/520 [00:14<00:00, 35.39it/s]


Test loss: 0.1346, test ROC-AUC:  0.774


Сведём результаты в таблицу.

| Model  | log_loss | roc_auc_score |
| ------ | -------- | ------------- |
|baseline| 0.1549   | 0.5           |
| linear | 0.1350   | 0.770         |
| FM     | 0.1346   | 0.774         |